In [1]:
pip install neo4j

In [2]:
#import required modules
from bs4 import BeautifulSoup
import requests as r
import os
import re
import ast
import json
import string
import openai
import psycopg2
import collections
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from neo4j import GraphDatabase
from jsonpath_ng.ext import parse
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

##LOAD WIKI DATA

In [19]:
#get request from wiki-site
url = 'https://en.wikipedia.org/wiki/List_of_disasters_in_the_United_States_by_death_toll'
response = r.get(url)
wiki_page_text = response.text
if response.status_code == 200:
    print("No Request error")

No Request error


In [20]:
#create a BesutifulSoup Object
soup = BeautifulSoup(wiki_page_text, 'html.parser')
#find the table we are looking for
table = soup.find('table',{'class':'wikitable sortable'})

In [21]:
print(soup)

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-toc-available" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of disasters in the United States by death toll - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-c

In [22]:
#EXTRACTING ALL THE KEYS
keys_list = []
#find all `tr` tags
table_data = table.find_all('tr')
#iterate over each table row to extract the <th>-tag
for i in table_data:
    key = i.find_all('th')
    keys = [ele.text.strip() for ele in key]
    if keys:
        keys_list.append(keys)

#EXTRACTING ALL THE VALUES
values_list = []
#find all `tr` tags
table_data = table.find_all('tr')
#iterate over each table row to extract the <td>-tag
for i in table_data:
    value = i.find_all('td')
    values = [ele.text.strip() for ele in value]
    if values:
        values_list.append(values)

#create dataframe with the extracted values
wiki_disasters_df = pd.DataFrame(values_list, columns=keys_list)

In [23]:
wiki_disasters_df

#need to edit this to find all tables within tag and append into one dataframe

,Fatalities,Year,Article,Type,Location,Damage (US$),Comments
0,"1,127,000+[1]",2020 – present,COVID-19 pandemic in the United States,Pandemic,Nationwide,see Economic impact of the COVID-19 pandemic i...,Fatalities estimated. Deadliest known disaster...


LOAD POSTGRES DATA

In [9]:
#establish connection to postgresdb
connection = psycopg2.connect(
        
    )

In [11]:
#set up cursor
cur = connection.cursor()

cur.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print(cur.fetchall())

[('nine_one_one',)]


In [18]:
cur.execute("select * FROM nine_one_one LIMIT 50")
lim = cur.fetchall()
columns = [desc[0] for desc in cur.description]

df = pd.DataFrame(lim, columns = columns)
df

for c in df.columns:
    print("---- %s ---" % c)
    print(df[c].value_counts())

---- sysident ---
sysident
1579395005    1
1579417238    1
1579402889    1
1579402949    1
1579416139    1
1579416159    1
1579416626    1
1579416657    1
1579416925    1
1579416936    1
1579416960    1
1579417202    1
1579417269    1
1579395818    1
1579417303    1
1579417504    1
1579417529    1
1579417549    1
1579417740    1
1579417758    1
1579417768    1
1579417786    1
1579417922    1
1579417924    1
1579402883    1
1579402852    1
1579400638    1
1579400583    1
1579395831    1
1579395846    1
1579395867    1
1579395923    1
1579395950    1
1579395961    1
1579395963    1
1579397545    1
1579397554    1
1579397555    1
1579398717    1
1579398722    1
1579398726    1
1579398751    1
1579399563    1
1579399573    1
1579399575    1
1579399707    1
1579400114    1
1579400133    1
1579400138    1
1579417931    1
Name: count, dtype: int64
---- cps ---
cps
LINK_TCS          22
TCS               19
LINK_TRDO          5
TRDO_ESME_Link     4
Name: count, dtype: int64
---- hno ---
hno
143

In [24]:
df

,sysident,cps,hno,glty,e1,stn,e4,transid,e8,lpri,...,psapid_hd,gday,fcof,cpf,mcn,ntme,nam,fcop,lat,psapid
0,1579395005,TCS,913,Handset AGPS,1a,MALL DR EAST-SW,820104,04008810ad,81fb,"Presentation Restricted User provided, not ver...",...,,04/15/2022,134.21,TMOB,DELTA TWP,21:19:28.755,T MOBILE USA,90%,42.741312,1260450001
1,1579395818,TRDO_ESME_Link,913,Handset GPS,1a,MALL DR EAST-SE,81B7,04000a25a1,81ae,"Presentation Allowed User provided, not verified",...,,04/15/2022,6.11,ATTMO AT&T Mob,DELTA TWP,21:19:47.728,AT&T Mobility,90%,42.741312,1260450001
2,1579395831,LINK_TCS,913,Network Unspecified,1a,MALL DR EAST-SW,820104,04009a5fe7,81fb,"Presentation Restricted User provided, not ver...",...,,04/15/2022,6.11,TMOB,DELTA TWP,21:19:52.419,T MOBILE USA,90%,42.741247,1260450001
3,1579395846,TCS,913,Network Unspecified,1a,MALL DR EAST-SW,820104,04008810c1,81fb,"Presentation Restricted User provided, not ver...",...,,04/15/2022,6.11,TMOB,DELTA TWP,21:19:52.479,T MOBILE USA,90%,42.741247,1260450001
4,1579395867,LINK_TCS,913,Network Unspecified,1a,MALL DR EAST-SW,820104,04009a5fe7,81fb,"Presentation Restricted User provided, not ver...",...,,04/15/2022,6.11,TMOB,DELTA TWP,21:19:52.419,T MOBILE USA,90%,42.741247,1260450001
5,1579395923,TCS,913,Network Unspecified,1a,MALL DR EAST-SW,820104,04008810c1,81fb,"Presentation Restricted User provided, not ver...",...,,04/15/2022,6.11,TMOB,DELTA TWP,21:19:52.460,T MOBILE USA,90%,42.741247,1260450001
6,1579395950,LINK_TCS,913,Network Unspecified,1a,MALL DR EAST-SW,820104,04009a5fee,81fb,"Presentation Restricted User provided, not ver...",...,,04/15/2022,6.11,TMOB,DELTA TWP,21:19:57.71,T MOBILE USA,90%,42.741247,1260450001
7,1579395961,TCS,913,Network Unspecified,1a,MALL DR EAST-SW,820104,04008810c4,81fb,"Presentation Restricted User provided, not ver...",...,,04/15/2022,6.11,TMOB,DELTA TWP,21:19:57.27,T MOBILE USA,90%,42.741247,1260450001
8,1579395963,LINK_TCS,913,Network Unspecified,1a,MALL DR EAST-SW,820104,04009a5fee,81fb,"Presentation Restricted User provided, not ver...",...,,04/15/2022,6.11,TMOB,DELTA TWP,21:19:57.71,T MOBILE USA,90%,42.741247,1260450001
9,1579397545,LINK_TCS,3215,Handset Unspecified,15,MT HOPE AVE,81F4,04009a600f,81eb,"Presentation Allowed User provided, not verified",...,,07/30/2019,,CMCST,LANSING,21:20:35.346,"GRACE,CONNIE",,42.709447,1260650001


LOAD GUN VIOLENCE DATA

In [47]:
pd.read_csv("export-7248b811-bd03-4dfd-902f-80d78d4eb0b8.csv")

,Incident ID,Incident Date,State,City Or County,Address,Victims Killed,Victims Injured,Suspects Killed,Suspects Injured,Suspects Arrested,Operations
0,2810693,"January 24, 2024",Louisiana,New Orleans,3400 block of Wall Blvd,1,3,0,0,0,NaN
1,2809998,"January 23, 2024",California,Adelanto,Shadow Mountain Rd and Lessing Ave,6,0,0,0,0,NaN
2,2807503,"January 21, 2024",New York,Bronx,4137 White Plains Rd,1,3,0,0,0,NaN
3,2807473,"January 21, 2024",Illinois,Tinley Park,7430 173rd Pl,4,0,0,0,1,NaN
4,2807511,"January 21, 2024",Texas,Katy,4643 Russet Leaf Tr,3,2,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1995,1905360,"January 16, 2021",Illinois,Kankakee,600 block of N Evergreen Ave,0,4,0,1,1,NaN
1996,1903058,"January 15, 2021",Pennsylvania,Philadelphia,5000 block of N 10th St,0,4,0,0,0,NaN
1997,1899819,"January 11, 2021",Florida,Miami,2491 NE 58th St,0,4,0,0,0,NaN
1998,1898261,"January 9, 2021",Illinois,Evanston,100 Asbury Ave,5,2,1,0,0,NaN


In [44]:
url = 'https://www.gunviolencearchive.org/query/99d3ff0d-76d2-4042-bf55-15417482432b/export_csv'
r = requests.get(url)
text = r.iter_lines()
reader = csv.reader(text, delimiter=',')

In [45]:
print(reader)

In [46]:
for row in reader:
    print(row)

Error: iterator should return strings, not bytes (the file should be opened in text mode)